In [12]:
# Value Deliveries Analysis with CrewAI — FIXED
# Comprehensive analysis system for customer value deliveries, contract renewals, and marketing insights


# =============================
# Cell 1: Install and Import Dependencies
# =============================
"""
First, install the required packages (run in a terminal / new cell):


pip install crewai pandas matplotlib seaborn plotly openpyxl
# If you use OpenAI models via LangChain, also:
pip install langchain-openai
"""


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')


# CrewAI imports
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from typing import Type, Dict, List
from pydantic import BaseModel, Field, PrivateAttr
import json


# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [13]:
# =============================
# Cell 2: Data Loading and Preprocessing
# =============================


def load_and_preprocess_data(csv_file_path):
    """
    Load and preprocess the delivery data CSV
    Expected columns: Delivery, Technologies, Value Type, Customer, Delivery Date, Contributors
    """
    try:
        df = pd.read_csv(csv_file_path)
        print(f"Data loaded successfully! Shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        # Clean column names (remove extra spaces)
        df.columns = df.columns.str.strip()
        # Convert Delivery Date to datetime
        if 'Delivery Date' in df.columns:
            df['Delivery Date'] = pd.to_datetime(df['Delivery Date'], errors='coerce')
        # Clean customer names
        if 'Customer' in df.columns:
            df['Customer'] = df['Customer'].str.strip()
        # Fill missing values
        df = df.fillna('')
        return df
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None


# Load your CSV file - update the path as needed
CSV_FILE_PATH = "value_deliveries.csv" # Update this path

df = load_and_preprocess_data(CSV_FILE_PATH)

# Display data overview
if df is not None:
    print("\n=== DATA OVERVIEW ===")
    print(f"Total deliveries: {len(df)}")
    print(f"Unique customers: {df['Customer'].nunique() if 'Customer' in df.columns else 'N/A'}")
    print(f"Date range: {df['Delivery Date'].min()} to {df['Delivery Date'].max()}" if 'Delivery Date' in df.columns else "No date info")
    print(f"Value types: {df['Value Type'].unique().tolist() if 'Value Type' in df.columns else 'N/A'}")
    print("\nFirst 3 rows:")
    print(df.head(3).to_string())

Data loaded successfully! Shape: (5, 6)
Columns: ['Delivery', 'Technologies', 'Value Type', 'Customer', 'Delivery Date', 'Contributors']

=== DATA OVERVIEW ===
Total deliveries: 5
Unique customers: 3
Date range: 2024-01-15 00:00:00 to 2024-02-28 00:00:00
Value types: ['production deploy', 'tool creation', 'poc project']

First 3 rows:
                                                                                                                                                                                                             Delivery                        Technologies         Value Type             Customer Delivery Date                          Contributors
0     Implemented automated CI/CD pipeline reducing deployment time by 80% and eliminating manual errors. Value: Faster releases and improved reliability. Measured by: deployment frequency and error rate reduction    Jenkins, Docker, AWS ECS, Python  production deploy            Acme Corp    2024-01-15  John Smith, Sara

In [14]:
# =============================
# Cell 3: Data Exploration and Visualization
# =============================

def create_data_visualizations(df):
    """
    Create comprehensive visualizations of the delivery data
    """
    if df is None or df.empty:
        print("No data available for visualization")
        return
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Deliveries by Customer', 'Deliveries by Value Type', 
                       'Deliveries Over Time', 'Contributors Analysis'),
        specs=[[{"type": "bar"}, {"type": "pie"}],
               [{"type": "scatter"}, {"type": "bar"}]]
    )
    
    # 1. Deliveries by Customer
    if 'Customer' in df.columns:
        customer_counts = df['Customer'].value_counts().head(10)
        fig.add_trace(
            go.Bar(x=customer_counts.index, y=customer_counts.values, name='Deliveries by Customer'),
            row=1, col=1
        )
    
    # 2. Deliveries by Value Type
    if 'Value Type' in df.columns:
        value_type_counts = df['Value Type'].value_counts()
        fig.add_trace(
            go.Pie(labels=value_type_counts.index, values=value_type_counts.values, name='Value Types'),
            row=1, col=2
        )
    
    # 3. Deliveries Over Time
    if 'Delivery Date' in df.columns:
        df_time = df.copy()
        df_time = df_time[df_time['Delivery Date'].notna()]
        df_time['YearMonth'] = df_time['Delivery Date'].dt.to_period('M')
        time_counts = df_time['YearMonth'].value_counts().sort_index()
        fig.add_trace(
            go.Scatter(x=time_counts.index.astype(str), y=time_counts.values, 
                      mode='lines+markers', name='Deliveries Timeline'),
            row=2, col=1
        )
    
    # 4. Contributors Analysis
    if 'Contributors' in df.columns:
        # Count individual contributors
        all_contributors = []
        for contributors_str in df['Contributors'].dropna():
            if contributors_str:
                contributors = [c.strip() for c in str(contributors_str).split(',')]
                all_contributors.extend(contributors)
        
        if all_contributors:
            contributor_counts = pd.Series(all_contributors).value_counts().head(10)
            fig.add_trace(
                go.Bar(x=contributor_counts.index, y=contributor_counts.values, 
                      name='Top Contributors'),
                row=2, col=2
            )
    
    fig.update_layout(height=800, title_text="Value Deliveries Dashboard")
    # Use browser renderer to avoid nbformat dependency in notebooks without nbformat
    fig.show(renderer="browser")
    
create_data_visualizations(df)

In [15]:
# =============================
# Cell 4: Define CrewAI Tools
# =============================

class CustomerDataTool(BaseTool):
    name: str = "customer_data_tool"
    description: str = "Analyzes delivery data for a specific customer"
    # Store DataFrame as a private attribute so Pydantic doesn't validate it
    _df: pd.DataFrame | None = PrivateAttr(default=None)

    def __init__(self, df: pd.DataFrame | None):
        super().__init__()
        self._df = df
    
    def _run(self, customer_name: str) -> str:
        """Get all delivery data for a specific customer"""
        if self._df is None:
            return "No data available"
        
        if 'Customer' not in self._df.columns:
            return "'Customer' column not found in data"
        
        customer_data = self._df[self._df['Customer'].str.contains(customer_name, case=False, na=False)]
        
        if customer_data.empty:
            return f"No deliveries found for customer: {customer_name}"
        
        # Safe checks for optional columns
        value_types = customer_data['Value Type'].unique().tolist() if 'Value Type' in customer_data.columns else []
        technologies = customer_data['Technologies'].unique().tolist() if 'Technologies' in customer_data.columns else []
        if 'Delivery Date' in customer_data.columns:
            date_range = f"{customer_data['Delivery Date'].min()} to {customer_data['Delivery Date'].max()}"
        else:
            date_range = "N/A"
        
        contributors = []
        if 'Contributors' in customer_data.columns:
            contributors = list(set([c.strip() for contrib in customer_data['Contributors'].dropna() 
                                     for c in str(contrib).split(',') if c.strip()]))
        
        summary = {
            'total_deliveries': len(customer_data),
            'value_types': value_types,
            'technologies': technologies,
            'date_range': date_range,
            'contributors': contributors
        }
        
        return json.dumps(summary, indent=2, default=str)

class DeliveryAnalysisTool(BaseTool):
    name: str = "delivery_analysis_tool"
    description: str = "Analyzes delivery descriptions to extract value metrics and business impact"
    
    def _run(self, delivery_text: str) -> str:
        """Analyze delivery text for value indicators"""
        if not delivery_text:
            return "No delivery description provided"
        
        # Keywords indicating different types of value
        value_indicators = {
            'efficiency': ['automated', 'streamlined', 'optimized', 'faster', 'reduced time'],
            'cost_savings': ['cost reduction', 'saved money', 'budget', 'roi', 'cheaper'],
            'performance': ['improved performance', 'faster', 'optimized', 'enhanced'],
            'scalability': ['scalable', 'expanded', 'growth', 'capacity'],
            'innovation': ['new feature', 'innovative', 'cutting-edge', 'first-time'],
            'compliance': ['compliant', 'security', 'audit', 'regulation']
        }
        
        found_values = []
        text_lower = delivery_text.lower()
        
        for category, keywords in value_indicators.items():
            if any(keyword in text_lower for keyword in keywords):
                found_values.append(category)
        
        metrics_found = bool(any(char in delivery_text for char in ['%', '$']) or 
                           any(word in text_lower for word in ['increased', 'decreased', 'improved', 'reduced']))
        
        return f"Value categories: {found_values}, Contains metrics: {metrics_found}, Word count: {len(delivery_text.split())}"
# =============================

In [16]:
# =============================
# Cell 5: Create Specialized Agents
# =============================

def create_analysis_agents(df):
    """
    Create specialized agents for different types of analysis.
    Tools now store df in PrivateAttr to avoid Pydantic field errors.
    """
    
    # Customer Data Analyst Agent
    customer_analyst = Agent(
        role='Customer Data Analyst',
        goal='Analyze delivery data by customer to identify patterns, value delivered, and relationship strength',
        backstory="""You are an expert customer success analyst with deep experience in analyzing 
        client relationships through delivery data. You excel at identifying customer engagement patterns, 
        value realization, and relationship health indicators.""",
        verbose=True,
        allow_delegation=False,
        tools=[CustomerDataTool(df), DeliveryAnalysisTool()]
    )
    
    # Contract Renewal Strategist Agent
    renewal_strategist = Agent(
        role='Contract Renewal Strategist',
        goal='Generate insights and strategies for customer contract renewals based on delivery history',
        backstory="""You are a senior account manager and renewal strategist with expertise in 
        customer retention. You analyze delivery patterns and customer satisfaction indicators to 
        create compelling renewal strategies and identify upselling opportunities focused on the deliveries. 
        You create a nice pitch selling text for renewing the contract, listing the deliveries e key points.""",
        verbose=True,
        allow_delegation=False,
        tools=[CustomerDataTool(df)]
    )
    
    # Marketing Intelligence Agent
    marketing_agent = Agent(
        role='Marketing Intelligence Specialist',
        goal='Create marketing insights and materials based on successful delivery patterns',
        backstory="""You are a marketing strategist specializing in B2B technology services. 
        You excel at analyzing successful project patterns to create compelling case studies, 
        identify market trends, and develop targeted marketing strategies for acquiring new clients based on the deliveries.""",
        verbose=True,
        allow_delegation=False,
        tools=[CustomerDataTool(df), DeliveryAnalysisTool()]
    )
    
    return customer_analyst, marketing_agent, renewal_strategist


In [17]:
# =============================
# Cell 6: Define Comprehensive Analysis Tasks for Contract Renewals
# =============================
def create_comprehensive_analysis_tasks_for_contract_renew(agents, customer_name=None):
    """
    Create a comprehensive set of tasks for analyzing delivery data
    """
    customer_analyst, marketing_agent, renewal_strategist = agents
    
    # Task 1: Customer-Specific Analysis
    if customer_name:
        customer_analysis_task = Task(
            description=f"""
            Conduct a comprehensive analysis of all deliveries for {customer_name}:
            
            1. Retrieve all delivery data for this customer
            2. Analyze the types of value delivered (production deploys, tools, POCs, etc.)
            3. Assess the technologies used and their evolution over time
            4. Evaluate the team members involved and their contributions
            5. Identify patterns in delivery frequency and project complexity
            6. Assess the overall relationship health and engagement level
            
            Provide a detailed customer profile with strengths, engagement patterns, and areas of concern.
            """,
            agent=customer_analyst,
            expected_output="Detailed customer analysis report with engagement patterns and relationship health assessment"
        )

    # Task 2: Contract Renewal Strategy
    renewal_strategy_task = Task(
        description=f"""
        Based on the customer analysis, develop contract renewal strategies:
        
        1. Identify key value propositions to emphasize in renewal discussions
        2. Recommend upselling opportunities based on delivery patterns
        3. Create risk mitigation strategies for at-risk renewals
        5. Develop success metrics and KPIs for renewal conversations
        6. Suggest optimal timing for renewal discussions
        7. Create a compelling pitch text for renewing the contract, listing the deliveries and key points.
        
        Focus on {customer_name if customer_name else 'top customers'} for detailed strategies.
        """,
        agent=renewal_strategist,
        expected_output="Comprehensive contract renewal strategy upselling opportunities based on the deliveries. Use the deliveries to back up your points and create nice arguments for renewing listing the delivery data.",
        context=[customer_analysis_task]
    )
    
    return [customer_analysis_task, renewal_strategy_task]


In [18]:
# =============================
# Cell 7: Define Comprehensive Analysis Tasks for Marketing
# =============================
def create_comprehensive_analysis_tasks_for_marketing(agents, customer_name=None):
    """
    Create a comprehensive set of tasks for analyzing delivery data
    """
    customer_analyst, marketing_agent, renewal_strategist = agents
    
    # Task 1: Customer-Specific Analysis
    if customer_name:
        customer_analysis_task = Task(
            description=f"""
            Conduct a comprehensive analysis of all deliveries for {customer_name}:
            
            1. Retrieve all delivery data for this customer
            2. Analyze the types of value delivered (production deploys, tools, POCs, etc.)
            3. Assess the technologies used and their evolution over time
            4. Evaluate the team members involved and their contributions
            5. Identify patterns in delivery frequency and project complexity
            6. Assess the overall relationship health and engagement level
            
            Provide a detailed customer profile with strengths, engagement patterns, and areas of concern.
            """,
            agent=customer_analyst,
            expected_output="Detailed customer analysis report with engagement patterns and relationship health assessment"
        )
    
    # Task 2: Marketing Intelligence
    marketing_intelligence_task = Task(
        description="""
        Generate marketing insights and materials based on successful delivery patterns:
        
        1. Identify the most successful project types and outcomes
        2. Create customer success story templates
        3. Identify target market segments based on successful engagements
        4. Develop value proposition messaging for different customer types
        5. Recommend marketing channels and approaches for new client acquisition
        6. Create competitive differentiation points.
        7. Create a marketing storytelling based on the deliveries.
        
        Focus on creating actionable marketing strategies and materials.
        """,
        agent=marketing_agent,
        expected_output="Marketing strategy storyteling of the deliveries so it can be used in marketing materials",
        context=[customer_analysis_task]
    )
    
    return [customer_analysis_task, marketing_intelligence_task]


In [19]:
# =============================
# Cell 8: Function Renew Analysis to Specific Customer 
# =============================

def analyze_customer_for_renew(df, customer_name, save_results=True):
    """
    Run comprehensive analysis for a specific customer
    """
    if df is None:
        print("No data available for analysis")
        return None
    
    print(f"\n🔍 Starting comprehensive analysis for customer: {customer_name}")
    
    # Check if customer exists
    customer_matches = df[df['Customer'].str.contains(customer_name, case=False, na=False)]
    if customer_matches.empty:
        print(f"❌ No deliveries found for customer: {customer_name}")
        available_customers = df['Customer'].unique()
        print(f"Available customers: {available_customers[:10]}...")  # Show first 10
        return None
    
    print(f"✅ Found {len(customer_matches)} deliveries for {customer_name}")
    
    # Create agents
    agents = create_analysis_agents(df)
    
    # Create tasks
    tasks = create_comprehensive_analysis_tasks_for_contract_renew(agents, customer_name)
    
    # Create and run crew
    crew = Crew(
        agents=list(agents),
        tasks=tasks,
        process=Process.sequential,
        verbose=True
    )
    
    try:
        print(f"🚀 Executing analysis crew...")
        result = crew.kickoff()
        
        if save_results:
            # Save results to file
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"results/customer_analysis_{customer_name.replace(' ', '_')}_{timestamp}.txt"
            with open(filename, 'w') as f:
                f.write(f"Renew Analysis - {customer_name}\n")
                f.write(f"Generated: {datetime.now()}\n")
                f.write("="*50 + "\n\n")
                f.write(str(result))
            print(f"📄 Results saved to: {filename}")
        
        return result
        
    except Exception as e:
        print(f"❌ Error during analysis: {e}")
        return None


In [9]:
# =============================
# Cell 9: Function Overall Marketing Analysis for Specific Customer
# =============================

def analyze_custumer_for_marketing(df, customer_name, save_results=True):
    """
    Run comprehensive marketting and business intelligence analysis
    """
    if df is None:
        print("No data available for analysis")
        return None
    
    print(f"\n📊 Starting comprehensive marketing and business intelligence analysis for {customer_name}...")
    
    # Create agents
    agents = create_analysis_agents(df)
    
    # Create tasks (without specific customer focus)
    tasks = create_comprehensive_analysis_tasks_for_marketing(agents, customer_name)
    
    # Create and run crew
    crew = Crew(
        agents=list(agents),
        tasks=tasks,
        process=Process.sequential,
        verbose=True
    )
    
    try:
        print("🚀 Executing market analysis crew...")
        result = crew.kickoff()
        
        if save_results:
            # Save results to file
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"results/marketing_analysis_{customer_name.replace(' ', '_')}_{timestamp}.txt"
            with open(filename, 'w') as f:
                f.write(f"Marketing Analysis - {customer_name}\n")
                f.write(f"Generated: {datetime.now()}\n")
                f.write("="*50 + "\n\n")
                f.write(str(result))
            print(f"📄 Results saved to: {filename}")
        
        return result
        
    except Exception as e:
        print(f"❌ Error during market analysis: {e}")
        return None


In [20]:
# =============================
# Cell 10: Interactive Analysis Runner
# =============================

def run_interactive_analysis(df, customer_name=None):
    """
    Interactive function to run different types of analysis
    """
    if df is None:
        print("❌ No data loaded. Please check your CSV file path.")
        return
    
    print("\n🎯 CREWAI DELIVERY ANALYSIS SYSTEM")
    print("="*50)
    print("Available analysis options:")
    print("1. Analyze specific customer")
    print("2. Run market trends analysis")
    print("3. Show available customers")
    print("4. Show data summary")
    
    # For notebook execution, we'll run both types of analysis
    print("\n📋 DATA SUMMARY:")
    print(f"Total deliveries: {len(df)}")
    print(f"Unique customers: {df['Customer'].nunique()}")

    if customer_name == None:
        print(f"Top customers by delivery count:")
        customer_counts = df['Customer'].value_counts().head(5)
        for customer, count in customer_counts.items():
            print(f"  • {customer}: {count} deliveries")

        top_customer = customer_counts.index[0]
        
        # Example: Analyze the top customer
        customer_result = None
        if not customer_counts.empty:
            print(f"\n🔍 Running renew analysis for top customer: {top_customer}")
            customer_result = analyze_customer_for_renew(df, top_customer, save_results=True)
            print(f"\n🔍 Running marketing analysis for top customer: {top_customer}")
            customer_result = analyze_custumer_for_marketing(df, top_customer, save_results=True)

    if customer_name:
        print(f"\n🔍 Running renew analysis for specified customer: {customer_name}")
        customer_result = analyze_customer_for_renew(df, customer_name, save_results=True)
        print(f"\n🔍 Running marketing analysis for specified customer: {customer_name}")
        customer_result = analyze_custumer_for_marketing(df, customer_name, save_results=True)
    
    return customer_result


In [21]:
# =============================
# Cell 11: Execute the Analysis
# =============================

"""
Run the comprehensive analysis system
"""
if df is not None:
    # Execute interactive analysis
    results = run_interactive_analysis(df)
    print("\n✅ Analysis completed! Check the generated report files.")
else:
    print("❌ Please update the CSV_FILE_PATH variable with your actual file path and run the cells again.")
    print("\nExpected CSV structure:")
    print("- Delivery: Description of what was delivered")
    print("- Technologies: Technologies used by development team")
    print("- Value Type: Type of value (production deploy, tool creation, POC, etc.)")
    print("- Customer: The client that received the value")
    print("- Delivery Date: Date of delivery")
    print("- Contributors: Team members who delivered the value")



🎯 CREWAI DELIVERY ANALYSIS SYSTEM
Available analysis options:
1. Analyze specific customer
2. Run market trends analysis
3. Show available customers
4. Show data summary

📋 DATA SUMMARY:
Total deliveries: 5
Unique customers: 3
Top customers by delivery count:
  • Acme Corp: 2 deliveries
  • TechStart Inc: 2 deliveries
  • Global Services Ltd: 1 deliveries

🔍 Running renew analysis for top customer: Acme Corp

🔍 Starting comprehensive analysis for customer: Acme Corp
✅ Found 2 deliveries for Acme Corp
🚀 Executing analysis crew...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 725e1e02-e89e-4cfb-bd4c-edf1cbc4ef88                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Data Analyst                                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│              Conduct a comprehensive analysis of all deliveries for Acme Corp:                                  │
│                                                                                                                 │
│              1. Retrieve all delivery data for this customer                                                    │
│              2. Analyze the types of value delivered (production deploys, tools, POCs, etc.)                    │
│              3. Assess the technologies used and their evolution over time                                      │
│              4. Evaluate the team members involved and their contributions                                      │
│              5. Identify patterns in delivery frequency and project complexity                                  │
│              6. Assess the overall relationship health and engagement level                                     │
│                                                                                                                 │
│              Provide a detailed customer profile with strengths, engagement patterns, and areas of concern.     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Data Analyst                                                                                   │
│                                                                                                                 │
│  Thought: I need to gather all delivery data for Acme Corp to conduct a comprehensive analysis.                 │
│                                                                                                                 │
│  Using Tool: customer_data_tool                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Marketing Intelligence Specialist                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Marketing Storytelling for Acme Corp**                                                                       │
│                                                                                                                 │
│  **Title: Powering Production with Innovation – A Success Story of Acme Corp**                                  │
│                                                                                                                 │
│  **Introduction:**                                                                                              │
│  At Acme Corp, we are committed to delivering cutting-edge technology solutions that drive real business        │
│  impact. Our recent engagements demonstrate our capability to adapt and innovate in the fast-paced tech         │
│  environment.                                                                                                   │
│                                                                                                                 │
│  **Delivery Overview:**                                                                                         │
│  In January and February 2024, Acme Corp experienced two significant deployments that showcased our strengths   │
│  in team collaboration and advanced technology integration. The involvement of a dynamic team led by experts    │
│  like John Smith, Mike Chen, Alex Rivera, Sarah Johnson, and David Park contributed to the successful           │
│  execution of these projects.                                                                                   │
│                                                                                                                 │
│  **Project Highlights:**                                                                                        │
│                                                                                                                 │
│  1. **First Delivery: Streamlining Processes with Jenkins and AWS**                                             │
│     - In our first deployment, we utilized Jenkins for continuous integration and AWS ECS for scalable cloud    │
│  hosting, coupled with robust Python applications. This strategic approach not only enhanced operational        │
│  efficiency but also ensured a seamless production environment.                                                 │
│                                                                                                                 │
│  2. **Second Delivery: Embracing Complexity with Kubernetes**                                                   │
│     - Building on the success of our first deployment, our second project embraced complexity through advanced  │
│  orchestrations using Docker and Kubernetes alongside Spring Boot and PostgreSQL. This initiative allowed Acme  │
│  Corp to elevate its application performance, catering to growing customer demands and setting a new standard   │
│  in operational excellence.                                                                                     │
│                                                                                                                 │
│  **Client Engagement:**                                                                                         │
│  The frequency of our deliveries—two impactful deployme

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f28f47d7-416b-4de7-b39c-4399039da824                                                                     │
│  Agent: Marketing Intelligence Specialist                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6e4ca490-2a42-4f04-9017-d39b067796c2                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Marketing Storytelling for Acme Corp**                                                         │
│                                                                                                                 │
│  **Title: Powering Production with Innovation – A Success Story of Acme Corp**                                  │
│                                                                                                                 │
│  **Introduction:**                                                                                              │
│  At Acme Corp, we are committed to delivering cutting-edge technology solutions that drive real business        │
│  impact. Our recent engagements demonstrate our capability to adapt and innovate in the fast-paced tech         │
│  environment.                                                                                                   │
│                                                                                                                 │
│  **Delivery Overview:**                                                                                         │
│  In January and February 2024, Acme Corp experienced two significant deployments that showcased our strengths   │
│  in team collaboration and advanced technology integration. The involvement of a dynamic team led by experts    │
│  like John Smith, Mike Chen, Alex Rivera, Sarah Johnson, and David Park contributed to the successful           │
│  execution of these projects.                                                                                   │
│                                                                                                                 │
│  **Project Highlights:**                                                                                        │
│                                                                                                                 │
│  1. **First Delivery: Streamlining Processes with Jenkins and AWS**                                             │
│     - In our first deployment, we utilized Jenkins for continuous integration and AWS ECS for scalable cloud    │
│  hosting, coupled with robust Python applications. This strategic approach not only enhanced operational        │
│  efficiency but also ensured a seamless production environment.                                                 │
│                                                                                                                 │
│  2. **Second Delivery: Embracing Complexity with Kubernetes**                                                   │
│     - Building on the success of our first deployment, our second project embraced complexity through advanced  │
│  orchestrations using Docker and Kubernetes alongside Spring Boot and PostgreSQL. This initiative allowed Acme  │
│  Corp to elevate its application performance, catering to growing customer demands and setting a new standard   │
│  in operational excellence.                                                                                     │
│                                                                                                                 │
│  **Client Engagement:**                               

📄 Results saved to: results/marketing_analysis_Acme_Corp_20250825_113747.txt

✅ Analysis completed! Check the generated report files.


In [ ]:
# =============================
# Cell 13: Utility Functions for Further Analysis
# =============================

def get_customer_insights(df, customer_name):
    """
    Quick customer insights without full CrewAI analysis
    """
    customer_data = df[df['Customer'].str.contains(customer_name, case=False, na=False)]
    
    if customer_data.empty:
        return f"No data found for customer: {customer_name}"
    
    insights = {
        'Total Deliveries': len(customer_data),
        'Value Types': customer_data['Value Type'].value_counts().to_dict() if 'Value Type' in customer_data.columns else {},
        'Most Recent Delivery': customer_data['Delivery Date'].max() if 'Delivery Date' in customer_data.columns else None,
        'Technologies Used': len([t.strip() for tech_str in customer_data['Technologies'].dropna() 
                                 for t in str(tech_str).split(',') if t.strip()]) if 'Technologies' in customer_data.columns else 0,
        'Contributors': len(set([c.strip() for contrib in customer_data['Contributors'].dropna() 
                                for c in str(contrib).split(',') if c.strip()])) if 'Contributors' in customer_data.columns else 0
    }
    
    return insights

def export_analysis_data(df, customer_name=None):
    """
    Export filtered data for external analysis
    """
    if customer_name:
        filtered_df = df[df['Customer'].str.contains(customer_name, case=False, na=False)]
        filename = f"deliveries_{customer_name.replace(' ', '_')}.xlsx"
    else:
        filtered_df = df
        filename = "all_deliveries_analysis.xlsx"
    
    # Create Excel file with multiple sheets
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        filtered_df.to_excel(writer, sheet_name='Raw Data', index=False)
        
        # Customer summary
        customer_summary = df.groupby('Customer').agg({
            'Delivery': 'count',
            'Value Type': lambda x: ', '.join(x.unique()),
            'Delivery Date': ['min', 'max'],
            'Contributors': lambda x: len(set([c.strip() for contrib in x.dropna() 
                                             for c in str(contrib).split(',') if c.strip()]))
        }).round(2)
        customer_summary.to_excel(writer, sheet_name='Customer Summary')
        
        # Technology trends
        if 'Technologies' in df.columns:
            all_techs = []
            for tech_str in df['Technologies'].dropna():
                if tech_str:
                    techs = [t.strip() for t in str(tech_str).split(',')]
                    all_techs.extend(techs)
            
            if all_techs:
                tech_summary = pd.Series(all_techs).value_counts().to_frame('Usage Count')
                tech_summary.to_excel(writer, sheet_name='Technology Trends')
    
    print(f"📊 Analysis data exported to: {filename}")
    return filename

# Example usage (uncomment to run specific analyses):
customer_insights = get_customer_insights(df, "TechStart Inc")
export_file = export_analysis_data(df)



In [ ]:
# =============================
# Cell 14: Utility Functions  for specific customer
# =============================

"""
Run the comprehensive analysis system
"""
if df is not None:
    # Execute interactive analysis
    results = run_interactive_analysis(df, customer_name="TechStart Inc")
    print("\n✅ Analysis completed! Check the generated report files.")
else:
    print("❌ Please update the CSV_FILE_PATH variable with your actual file path and run the cells again.")
    print("\nExpected CSV structure:")
    print("- Delivery: Description of what was delivered")
    print("- Technologies: Technologies used by development team")
    print("- Value Type: Type of value (production deploy, tool creation, POC, etc.)")
    print("- Customer: The client that received the value")
    print("- Delivery Date: Date of delivery")
    print("- Contributors: Team members who delivered the value")